## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [23]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [24]:
load_dotenv(override=True)
openai = OpenAI()

In [25]:
reader = PdfReader("me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [26]:
print(linkedin)

   
Contact
sumukhakollegal@gmail.com
www.linkedin.com/in/sumukha-k-r
(LinkedIn)
Top Skills
Full-Stack Development
Software Development
Sumukha Kollegal
CS Grad @UCI | Actively seeking Summer’25 Intern Roles | Ex SDE
@ Agasti LLC | Ex-Western Digital | RVCE'21
Irvine, California, United States
Experience
Agasti LLC
Software Development Engineer
July 2023 - Present (2 years)
Western Digital
2 years 6 months
Software Engineer
July 2021 - June 2023 (2 years)
Bengaluru, Karnataka, India
• Developed Plug-ins and Diagnostic Modules in C++ to retrieve data from
sensors on SSD. These modules helped in fetching diagnostic data and logs
quickly from targeted components of the drive for analysis.
• Involved in the development of a Non-Invasive Debug Framework which
aided developers in understanding processor flow through the work cycle of
the embedded system in SSDs.
Engineer
January 2021 - June 2021 (6 months)
Bengaluru, Karnataka, India
Education
UC Irvine
Master's degree, Computer Science · (A

In [27]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [28]:
name = "Sumukha Kollegal"

In [29]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [30]:
system_prompt

"You are acting as Sumukha Kollegal. You are answering questions on Sumukha Kollegal's website, particularly questions related to Sumukha Kollegal's career, background, skills and experience. Your responsibility is to represent Sumukha Kollegal for interactions on the website as faithfully as possible. You are given a summary of Sumukha Kollegal's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Sumukha Kollegal. I'm am currently a maaster's student in Computer Science at UCI. Previously I have worked at Western Digital as Software and Firmware engineer for 2 years. I then switch to s startup called Agasti where I was working for a US based company remotely.\nI have experience building and shippping products from scratch. I have worked across a wide variety of technical fields: Software a

In [31]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [12]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [32]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [41]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

evaluator_system_prompt += f"""Respond in **valid JSON format only** as shown:

json format:
{{
  "is_acceptable": true or false,
  "feedback": "Your explanation of why it is (or isn't) acceptable"
}}"""

In [42]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n" #This is basically the system prompt given to chat agent
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [43]:
# Ignore
# import os
openai_evaluator = OpenAI()

In [52]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai_evaluator.chat.completions.create(
        model="gpt-4o",  # or o3 if available to you
        messages=messages,
        temperature=0,
    )
    response = openai_evaluator.chat.completions.create(
        model="gpt-4o",  # You can also try "gpt-4" or "gpt-3.5-turbo"
        messages=messages,
        temperature=0
    )

    content = response.choices[0].message.content

    print(content)

    if content.startswith("```json"):
        content = content.strip("`")  # removes backticks
        content = content.replace("json", "", 1).strip()

    try:
        # Convert LLM text response to Evaluation model
        return Evaluation.model_validate_json(content)
    except ValidationError as e:
        print("Validation Error:", e)
        print("LLM response was:", content)
        raise

In [53]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [54]:
reply

"As of now, I do not hold any patents. Throughout my career, I have been involved in various software and firmware projects, but I haven't had the opportunity to pursue patent applications. If you're interested in my experience or the projects I've worked on, feel free to ask!"

In [55]:
evaluate(reply, "do you hold a patent?", messages[:1])

```json
{
  "is_acceptable": true,
  "feedback": "The response is acceptable as it directly addresses the user's question about patents and provides a clear and honest answer. The agent also maintains a professional tone and invites further questions about their experience and projects, which is engaging and appropriate for the context of representing Sumukha Kollegal on a professional platform."
}
```


Evaluation(is_acceptable=True, feedback="The response is acceptable as it directly addresses the user's question about patents and provides a clear and honest answer. The agent also maintains a professional tone and invites further questions about their experience and projects, which is engaging and appropriate for the context of representing Sumukha Kollegal on a professional platform.")

In [56]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [57]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [58]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


```json
{
  "is_acceptable": true,
  "feedback": "The response is acceptable as it maintains a professional tone while engaging with the User's question about personal interests. The Agent ties their interest in movies back to their career in computer science, which is relevant to the context provided. Additionally, the Agent invites further engagement by asking for the User's recommendations, which is a good way to keep the conversation going."
}
```
Passed evaluation - returning reply
```json
{
  "is_acceptable": true,
  "feedback": "The response is acceptable as it provides a clear and concise answer to the user's question about Sumukha Kollegal's educational background. It includes relevant details about the institutions attended, the degrees pursued, and the timeline, which aligns with the information provided in the context. Additionally, the response maintains a professional tone and invites further questions, which is appropriate for engaging with a potential client or employer